In [139]:
import csv

import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report


RANDOM_SEED = 42

In [140]:
dataset = 'model/data/landmarks.csv'
modelPath = 'model/landmarkIdentifier/landmarkIdentifier.hdf5'
tfPath = 'model/landmarkIdentifier/landmarkIdentifier.tflite'

NUM_CLASSES = 28

In [141]:
X_dataset = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, (21 * 2) + 1)))
y_dataset = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))

In [142]:
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.75, random_state=RANDOM_SEED)

In [143]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(21 * 2, )),
    tf.keras.layers.Dropout(0.1),  # Slightly reduced to prevent underfitting
    tf.keras.layers.Dense(64, activation='relu'),  # Increased units for more complexity
    tf.keras.layers.Dropout(0.3),  # Adjusted for balance between complexity and regularization
    tf.keras.layers.Dense(32, activation='relu'),  # Added another layer for depth
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

In [144]:
cp_callback = ModelCheckpoint(
    modelPath, verbose=1, save_best_only=True, monitor='val_loss')

# EarlyStopping to halt training when validation loss stops improving
es_callback = EarlyStopping(patience=10, verbose=1, restore_best_weights=True, monitor='val_loss')

# ReduceLROnPlateau to decrease the learning rate when a plateau is detected
lr_callback = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, verbose=1, min_lr=0.0001)

callbacks = [cp_callback, es_callback, lr_callback]

In [145]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [146]:
history = model.fit(
    X_train,
    y_train,
    epochs=1000,
    batch_size=64,
    validation_data=(X_test, y_test),
    callbacks=callbacks
)

Epoch 1/1000
46/56 [=======================>......] - ETA: 0s - loss: 3.2650 - accuracy: 0.0747 
Epoch 1: val_loss improved from inf to 3.01762, saving model to model/landmarkIdentifier/landmarkIdentifier.hdf5
56/56 [==============================] - 1s 4ms/step - loss: 3.2413 - accuracy: 0.0804 - val_loss: 3.0176 - val_accuracy: 0.1621 - lr: 0.0010
Epoch 2/1000
43/56 [======================>.......] - ETA: 0s - loss: 2.8968 - accuracy: 0.1766
Epoch 2: val_loss improved from 3.01762 to 2.44058, saving model to model/landmarkIdentifier/landmarkIdentifier.hdf5
56/56 [==============================] - 0s 2ms/step - loss: 2.8344 - accuracy: 0.1866 - val_loss: 2.4406 - val_accuracy: 0.3065 - lr: 0.0010
Epoch 3/1000
39/56 [===================>..........] - ETA: 0s - loss: 2.3792 - accuracy: 0.2752

/home/farhad/miniconda3/envs/tensorflow/lib/python3.9/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 3: val_loss improved from 2.44058 to 1.90557, saving model to model/landmarkIdentifier/landmarkIdentifier.hdf5
56/56 [==============================] - 0s 2ms/step - loss: 2.3081 - accuracy: 0.2948 - val_loss: 1.9056 - val_accuracy: 0.5467 - lr: 0.0010
Epoch 4/1000
49/56 [=========================>....] - ETA: 0s - loss: 1.9335 - accuracy: 0.3951
Epoch 4: val_loss improved from 1.90557 to 1.52195, saving model to model/landmarkIdentifier/landmarkIdentifier.hdf5
56/56 [==============================] - 0s 2ms/step - loss: 1.9174 - accuracy: 0.3962 - val_loss: 1.5220 - val_accuracy: 0.6324 - lr: 0.0010
Epoch 5/1000
46/56 [=======================>......] - ETA: 0s - loss: 1.6921 - accuracy: 0.4501
Epoch 5: val_loss improved from 1.52195 to 1.26054, saving model to model/landmarkIdentifier/landmarkIdentifier.hdf5
56/56 [==============================] - 0s 2ms/step - loss: 1.6661 - accuracy: 0.4579 - val_loss: 1.2605 - val_accuracy: 0.6698 - lr: 0.0010
Epoch 6/1000
47/56 [==========

In [147]:
val_loss, val_acc = model.evaluate(X_test, y_test, batch_size=128)

10/10 [==============================] - 0s 1ms/step - loss: 0.0700 - accuracy: 0.9873


In [148]:
model = tf.keras.models.load_model(modelPath)

In [149]:
model.save(modelPath, include_optimizer=False)

In [150]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

open(tfPath, 'wb').write(tflite_quantized_model)

INFO:tensorflow:Assets written to: /tmp/tmp35nuzx7e/assets


INFO:tensorflow:Assets written to: /tmp/tmp35nuzx7e/assets
2024-04-04 16:40:55.954932: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2024-04-04 16:40:55.954965: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2024-04-04 16:40:55.955149: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp35nuzx7e
2024-04-04 16:40:55.955900: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2024-04-04 16:40:55.955918: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /tmp/tmp35nuzx7e
2024-04-04 16:40:55.957961: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2024-04-04 16:40:55.987635: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path: /tmp/tmp35nuzx7e
2024-04-04 16:40:55.996152: I tensorflow/cc/saved_model/loader.cc:314] SavedModel

11104